# Skin Tumor Classification using Image and Tabular data

In [54]:
import os
import numpy as np
import pandas as pd
import skimage
from skimage import io,transform
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms, utils

### Data
The dataset is a combination of skin tumor image data and the sex and (grouped) age. Some sex and age records are missing (noted as "unknown"). The meta-data looks like the following as a few examples.

In [55]:
pd.read_csv('data/ISIC-2017_Training_Data_metadata.csv').sample(10)

,image_id,age_approximate,sex,class,melanoma,seborrheic_keratosis,nevus
1684,ISIC_0014062.jpg,60,female,0,1,0,0
1740,ISIC_0014253.jpg,60,female,2,0,0,1
1043,ISIC_0011382.jpg,85,male,2,0,0,1
533,ISIC_0006711.jpg,15,male,2,0,0,1
537,ISIC_0006940.jpg,15,male,2,0,0,1
487,ISIC_0002469.jpg,15,female,2,0,0,1
1149,ISIC_0012386.jpg,85,male,1,0,1,0
1050,ISIC_0011402.jpg,20,male,2,0,0,1
1720,ISIC_0014169.jpg,85,male,1,0,1,0
427,ISIC_0000525.jpg,unknown,unknown,2,0,0,1


First we define the dataset for training and validation below.

In [56]:
class dataset_lesion(Dataset):
    def __init__(self,csv_file,img_dir,transform=True,dropna=True):
        self.csv_file = csv_file
        self.img_dir = img_dir
        self.transform = transform
        self.dropna = dropna
        if self.dropna:
            missing_values =['unknown','']
            self.meta_data = pd.read_csv(csv_file,na_values=missing_values).dropna()
            self.meta_data['sex']= pd.get_dummies(self.meta_data['sex'])
        else:
            self.meta_data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.meta_data)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = os.path.join(self.img_dir,self.meta_data.iloc[idx,0])
        img = io.imread(img_path)
        if self.transform:
            img = transform.resize(img,(224,224))
            transform_tensor = transforms.Compose([transforms.ToTensor(),
                                                  transforms.RandomRotation(30),
                                                  transforms.RandomVerticalFlip(),
                                                  transforms.RandomHorizontalFlip()])
            img = transform_tensor(img)
        
        target = torch.from_numpy(np.array(self.meta_data.iloc[idx,3]).astype(float))
        
        if self.dropna:
            tab_data = torch.from_numpy(np.array(self.meta_data.iloc[idx,1:3]).astype(float))
            sample = {'image':img,'tab_data':tab_data,'target':target}
        else:
            sample = {'image':img,'target':target}
                
        return sample        

In [57]:
dataset = dataset_lesion(csv_file='data/ISIC-2017_Training_Data_metadata.csv',
                         img_dir='data/ISIC-2017_Training_Data',dropna=False)
rand1 = np.random.randint(len(dataset))
sample_image,sample_target =dataset[rand1]['image'],dataset[rand1]['target']
print(f'Number of data points: {len(dataset)}; Image size: {sample_image.shape}; sample target: {sample_target};')

Number of data points: 2000; Image size: torch.Size([3, 224, 224]); sample target: 2.0;


The training/validation data split is 80%/20%.

In [58]:
num_train_data = int(len(dataset)*0.8)
train_data,valid_data=torch.utils.data.random_split(dataset, [num_train_data,len(dataset)-num_train_data])

In [59]:
train_loader=DataLoader(train_data,shuffle=True,batch_size=20)
valid_loader=DataLoader(valid_data,shuffle=False,batch_size=20)

In [60]:
di=iter(train_loader)
di.next()['image'].shape

torch.Size([20, 3, 224, 224])

Creating dataset with both image and tabular data (sex and age group) below.

In [61]:
dataset_2 = dataset_lesion(csv_file='data/ISIC-2017_Training_Data_metadata.csv'
                                 ,img_dir='data/ISIC-2017_Training_Data')
num_train_data_2 = int(len(dataset_2)*0.8)
train_data_2,valid_data_2=torch.utils.data.random_split(dataset_2, [num_train_data_2,len(dataset_2)-num_train_data_2])

train_loader_2=DataLoader(train_data_2,shuffle=True,batch_size=20)
valid_loader_2=DataLoader(valid_data_2,shuffle=False,batch_size=20)

In [62]:
di_2=iter(train_loader_2)
di_2.next()['image'].shape

torch.Size([20, 3, 224, 224])

### Model
A transfer learning approach is taken by using pre-trained model ResNet_50 replacing the final fully connected layer with customized fully connected layer with three outputs. The loss function is defined as multi-class cross-entropy loss with logits and Adam as the optimizer. 

In [63]:
use_cuda = True if torch.cuda.is_available() else False

In [64]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
weight_class_loss = torch.tensor([0.45,0.35,0.2])
model = torchvision.models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad=False

model.fc=torch.nn.Linear(in_features=2048, out_features=3,bias=True)

if use_cuda:
    model.cuda()
    weight_class_loss=weight_class_loss.cuda()
    
criterion = torch.nn.CrossEntropyLoss(weight=weight_class_loss)
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max',factor = 0.25,patience=0,verbose=True)

The cross-entropy loss is weighted higher to the first and second class because their accuracy is lower but more important to detect correctly.

The following creates a customized model combining the pre-trained Resnet_50 for image data and a couple of fully connected layers for tabular data (sex and age group). 

In [73]:
class model_img_tab(torch.nn.Module):
    def __init__(self):
        super(model_img_tab, self).__init__()
        self.tanh = torch.nn.Tanh()
        self.fc1 = torch.nn.Linear(2,64)
        self.fc2 = torch.nn.Linear(64,16)
        # image features of 2048 + 16 tabular features 
        self.fc3 = torch.nn.Linear(2064, 3,bias=True)
        
        resnet50 = torchvision.models.resnet50(pretrained=True)
        self.img_cnn = torch.nn.Sequential(*list(resnet50.children())[:-1])
        for param in self.img_cnn.parameters():
            param.requires_grad=False
                
    def forward(self,img,tab):
        tab = self.tanh(self.fc1(tab))
        tab = self.fc2(tab)
        img = torch.squeeze(self.img_cnn(img))
        out = self.fc3(torch.cat((img,tab),dim=1))        
        return out

In [74]:
model_img_tab = model_img_tab()
weight_class_loss = torch.tensor([0.45,0.35,0.2])

if use_cuda:
    model_img_tab.cuda() 
    weight_class_loss=weight_class_loss.cuda()
    
criterion_img_tab = torch.nn.CrossEntropyLoss(weight=weight_class_loss)
optimizer_img_tab = torch.optim.Adam(model_img_tab.parameters(), lr=0.001)
scheduler_img_tab = ReduceLROnPlateau(optimizer_img_tab, mode='max',factor = 0.1,patience=0,verbose=True)

### Training and Validation
The training function for image only model is defined below.

In [67]:
def train(n_epochs, train_loader,valid_loader, model, optimizer,scheduler, criterion, use_cuda, save_path):
    for e in range(1,n_epochs+1):
        train_loss = 0
        valid_loss = 0
        valid_loss_min = np.Inf
        
        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            img = data['image'].float()
            #tab_data = data['tab_data'][:,age0_sex1].float()
            target = data['target'].long()
            if use_cuda:
                img,target = img.cuda(),target.cuda()

            pred = torch.squeeze(model(img))
            loss = criterion(pred,target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (i + 1)) * (loss.item() - train_loss))

            if i % 10 == 9:    # print training loss every specified number of mini-batches
                print('Epoch %d, Batch %d training loss: %.6f' % (e, i + 1, train_loss))

        model.eval()
        for i, data in enumerate(valid_loader):
            img = data['image'].float()
            #tab_data = data['tab_data'][:,age0_sex1].float()
            target = data['target'].long()
            if use_cuda:
                img,target = img.cuda(),target.cuda()

            pred = torch.squeeze(model(img))
            loss = criterion(pred,target)
            valid_loss = valid_loss + ((1 / (i + 1)) * (loss.item() - valid_loss))

            # print training/validation statistics 
        print('Epoch {}: Validation Loss: {:.6f}'.format(e,valid_loss))
        scheduler.step(valid_loss)

        if valid_loss < valid_loss_min:
            valid_loss_min = valid_loss
            torch.save(model.state_dict(),save_path)
        
    return model    

The image and tabular data combined model is defined below.

In [76]:
def train_img_tab(n_epochs, train_loader,valid_loader, model, optimizer,scheduler, criterion, use_cuda, save_path):
    for e in range(1,n_epochs+1):
        train_loss = 0
        valid_loss = 0
        valid_loss_min = np.Inf
        
        model.train()
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            img = data['image'].float()
            tab = data['tab_data'].float()
            target = data['target'].long()
            if use_cuda:
                img,tab,target = img.cuda(),tab.cuda(),target.cuda()

            pred = torch.squeeze(model(img,tab))
            loss = criterion(pred,target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (i + 1)) * (loss.item() - train_loss))

            if i % 10 == 9:    # print training loss every specified number of mini-batches
                print('Epoch %d, Batch %d training loss: %.6f' % (e, i + 1, train_loss))

        model.eval()
        for i, data in enumerate(valid_loader):
            img = data['image'].float()
            tab = data['tab_data'].float()
            target = data['target'].long()
            if use_cuda:
                img,tab,target = img.cuda(),tab.cuda(),target.cuda()

            output = torch.squeeze(model(img,tab))
            loss = criterion(output,target)
            valid_loss = valid_loss + ((1 / (i + 1)) * (loss.item() - valid_loss))
            
            # print training/validation statistics 
        print('Epoch {}: Validation Loss: {:.6f}'.format(e,valid_loss))
        scheduler.step(valid_loss)

        if valid_loss < valid_loss_min:
            valid_loss_min = valid_loss
            torch.save(model.state_dict(),save_path)
        
    return model 

The following shows the training progress with 30 epochs for the image only model.

In [69]:
train(30, train_loader,valid_loader, model, optimizer,scheduler, criterion, use_cuda=True, save_path='model.pt')

Epoch 1, Batch 10 training loss: 1.234344
Epoch 1, Batch 20 training loss: 1.175181
Epoch 1, Batch 30 training loss: 1.092892
Epoch 1, Batch 40 training loss: 1.040149
Epoch 1, Batch 50 training loss: 1.014026
Epoch 1, Batch 60 training loss: 0.995923
Epoch 1, Batch 70 training loss: 0.972791
Epoch 1, Batch 80 training loss: 0.959124
Epoch 1: Validation Loss: 0.899880
Epoch 2, Batch 10 training loss: 0.978761
Epoch 2, Batch 20 training loss: 0.907497
Epoch 2, Batch 30 training loss: 0.932743
Epoch 2, Batch 40 training loss: 0.911808
Epoch 2, Batch 50 training loss: 0.908966
Epoch 2, Batch 60 training loss: 0.892483
Epoch 2, Batch 70 training loss: 0.875453
Epoch 2, Batch 80 training loss: 0.871224
Epoch 2: Validation Loss: 0.805278
Epoch     2: reducing learning rate of group 0 to 2.5000e-04.
Epoch 3, Batch 10 training loss: 0.825322
Epoch 3, Batch 20 training loss: 0.795031
Epoch 3, Batch 30 training loss: 0.761040
Epoch 3, Batch 40 training loss: 0.761382
Epoch 3, Batch 50 training l

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Here is the performance on validation data using the image only model. Class 3 has a high score while class 1 is no better than random guess even with the weighting over class 1 and 2. 

In [70]:
model.load_state_dict(torch.load('model.pt'))
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
classes={0:'Melanoma',1:'seborrheic',2:'nevus'}

model.eval()
with torch.no_grad():
    for i, data in enumerate(valid_loader):
        img = data['image'].float()
        #tab_data = data['tab_data'][:,age0_sex1].float()
        target = data['target'].long()
        batch_size = len(target)
        if use_cuda:
            img,target = img.cuda(),target.cuda()

        output = torch.squeeze(model(img))
        loss = criterion(output,target)

        _, pred = torch.max(output,1)
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not use_cuda else np.squeeze(correct_tensor.cpu().numpy())

        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] +=1

for i in range(3):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
np.sum(class_correct), np.sum(class_total)))

Test Accuracy of Melanoma: 31% (23/74)
Test Accuracy of seborrheic: 61% (33/54)
Test Accuracy of nevus: 86% (234/272)

Test Accuracy (Overall): 72% (290/400)


The following is the training for the image and tabular combined model.

In [77]:
train_img_tab(30, train_loader_2,valid_loader_2, model_img_tab, optimizer_img_tab, scheduler_img_tab,criterion_img_tab, use_cuda=True, save_path='model_img_tab.pt')

Epoch 1, Batch 10 training loss: 0.762827
Epoch 1, Batch 20 training loss: 0.821468
Epoch 1, Batch 30 training loss: 0.814042
Epoch 1, Batch 40 training loss: 0.820225
Epoch 1, Batch 50 training loss: 0.830576
Epoch 1, Batch 60 training loss: 0.838275
Epoch 1, Batch 70 training loss: 0.835455
Epoch 1: Validation Loss: 0.944135
Epoch 2, Batch 10 training loss: 0.835027
Epoch 2, Batch 20 training loss: 0.861178
Epoch 2, Batch 30 training loss: 0.848038
Epoch 2, Batch 40 training loss: 0.832284
Epoch 2, Batch 50 training loss: 0.827601
Epoch 2, Batch 60 training loss: 0.826626
Epoch 2, Batch 70 training loss: 0.810804
Epoch 2: Validation Loss: 0.876420
Epoch     2: reducing learning rate of group 0 to 1.0000e-04.
Epoch 3, Batch 10 training loss: 0.743822
Epoch 3, Batch 20 training loss: 0.744640
Epoch 3, Batch 30 training loss: 0.706057
Epoch 3, Batch 40 training loss: 0.695929
Epoch 3, Batch 50 training loss: 0.701045
Epoch 3, Batch 60 training loss: 0.704566
Epoch 3, Batch 70 training l

model_img_tab(
  (tanh): Tanh()
  (fc1): Linear(in_features=2, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=2064, out_features=3, bias=True)
  (img_cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), 

The model is assessed using the validation data below. The tabular data seems to have a slightly better performance for class 1, although the results need to be confirmed more testing.

In [78]:
model_img_tab.load_state_dict(torch.load('model_img_tab.pt'))
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
classes={0:'Melanoma',1:'seborrheic',2:'nevus'}

model_img_tab.eval()
with torch.no_grad():
    for i, data in enumerate(valid_loader_2):
        img = data['image'].float()
        tab_data = data['tab_data'].float()
        target = data['target'].long()
        batch_size = len(target)
        if use_cuda:
            img,tab_data,target = img.cuda(),tab_data.cuda(),target.cuda()

        output = torch.squeeze(model_img_tab(img,tab_data))

        _, pred = torch.max(output,1)
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not use_cuda else np.squeeze(correct_tensor.cpu().numpy())

        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] +=1

for i in range(3):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
np.sum(class_correct), np.sum(class_total)))

Test Accuracy of Melanoma: 41% (28/68)
Test Accuracy of seborrheic: 54% (28/51)
Test Accuracy of nevus: 83% (191/230)

Test Accuracy (Overall): 70% (247/349)
